In [1]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

from ultralytics import YOLO

from PIL import Image, ImageDraw

Ultralytics YOLOv8.2.27 🚀 Python-3.8.5 torch-2.2.2 CPU (Apple M1)
Setup complete ✅ (8 CPUs, 16.0 GB RAM, 401.7/460.4 GB disk)


#### Original YoloV8 model

In [41]:
model_original = YOLO('yolov8n.pt')  # load a pretrained YOLOv8n detection model

#### Pre-trained on boat data YoloV8 model

In [61]:
model = YOLO("weights/best.pt")

#### Path to the image

In [111]:
image_path = "googleEarth/lerins2015-06-17.png"

In [112]:
save_image_path = image_path.split("/")[1].split(".")[0]

### Square crop and resize the image

In [114]:
image = Image.open(image_path)

width, height = image.size

min_side = min(width, height)
left = (width - min_side) / 2
top = (height - min_side) / 2
right = (width + min_side) / 2
bottom = (height + min_side) / 2
image_cropped = image.crop((left, top, right, bottom))

image_resized = image_cropped.resize((512, 512))

image_resized.save(f"cropped_image_{save_image_path}.png")

### Detect boats

In [115]:
pred = model(image_resized)


0: 1504x1504 38 MBs, 25 SBs, 2611.7ms
Speed: 37.3ms preprocess, 2611.7ms inference, 2.4ms postprocess per image at shape (1, 3, 1504, 1504)


In [116]:
pred[0].show()

In [117]:
pred[0].save(filename=f"results_cropped_{save_image_path}.jpg")

'results_cropped_lerins2015-06-17.jpg'

In [46]:
pred[0].boxes.xyxy

tensor([[1427.0275, 1025.2297, 1484.4053, 1106.8408],
        [ 143.2024, 1153.6144,  216.0581, 1254.2140],
        [1626.2706, 1001.4590, 1684.6981, 1075.0027],
        [1742.6804, 1026.2489, 1846.2625, 1131.8687]])

## Create mask matrix

In [118]:
bboxes = pred[0].boxes.xyxy.cpu().numpy()  

image = image_resized
width, height = image.size

mask = Image.new('RGB', (width, height), color='black')
draw = ImageDraw.Draw(mask)

for box in bboxes:
    x1, y1, x2, y2 = box
    draw.rectangle([x1, y1, x2, y2], fill='white')

mask.save(f"mask_image_{save_image_path}.png")
mask.show()

### Add margin to the bounding box

In [53]:
bboxes = pred[0].boxes.xyxy.cpu().numpy()  

image = Image.open(image_path)
width, height = image.size

mask = Image.new('RGB', (width, height), color='black')
draw = ImageDraw.Draw(mask)

margin = 20

for box in bboxes:
    x1, y1, x2, y2 = box
    x1 = max(0, x1 - margin)
    y1 = max(0, y1 - margin)
    x2 = min(width, x2 + margin)
    y2 = min(height, y2 + margin)
    draw.rectangle([x1, y1, x2, y2], fill='white')
    
mask.save(f"margin_mask_image_{image_path}.png")
mask.show()